<a href="https://colab.research.google.com/github/raflins/Atividade_Teoria_Filas/blob/main/Atividade_Teoria_Filas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import heapq
from datetime import datetime, timedelta

# --- Configurações Globais ---
DURACAO_CONSUTA_MIN = 20
MAPEAMENTO_PRIORIDADE = {
    'vermelho': 4,
    'laranja': 3,
    'amarelo': 2,
    'azul': 1,
}
LISTA_ESPECIALIDADES = [
    'Clínica Geral', 'Pediatria', 'Ortopedia', 'Cardiologia',
    'Ginecologia/Obstetrícia', 'Otorrinolaringologia'
]

# --- Estruturas de Dados ---
filas_especialidades = {espec: [] for espec in LISTA_ESPECIALIDADES}
proxima_sequencia_paciente = 0  # Contador de ordem de chegada
ultimo_fim_consulta = {espec: datetime(1, 1, 1, 8, 0) for espec in LISTA_ESPECIALIDADES}  # Hora inicial fictícia (08:00)

def string_para_datahora(string_hora):
    """Converte string 'HH:MM' para objeto datetime (data de hoje)."""
    hoje = datetime.now().date()
    hora, minuto = map(int, string_hora.split(':'))
    return datetime(hoje.year, hoje.month, hoje.day, hora, minuto)

def duracao_para_minutos(duracao):
    """Converte timedelta para minutos totais."""
    return duracao.total_seconds() / 60

# --- Classe Paciente ---
class Paciente:
    def __init__(self, nome_completo, idade, cor_urgencia, especialidade, hora_chegada_str, sequencia_chegada):
        self.nome_completo = nome_completo
        self.idade = idade
        self.cor_urgencia = cor_urgencia
        self.especialidade = especialidade
        self.hora_chegada = string_para_datahora(hora_chegada_str)
        self.prioridade = MAPEAMENTO_PRIORIDADE.get(cor_urgencia.lower(), 1)
        self.sequencia_chegada = sequencia_chegada
        self.inicio_previsto = None
        self.fim_previsto = None

    def __repr__(self):
        return f"({self.nome_completo}, {self.cor_urgencia[:3]}, Chegada: {self.hora_chegada.strftime('%H:%M')})"

# --- Funções Operacionais ---

def adicionar_paciente(nome_completo, idade, cor_urgencia, especialidade, hora_chegada_str):
    """
    Insere paciente na fila de prioridade para a especialidade dada. O(log n_s).
    """
    global proxima_sequencia_paciente
    if especialidade not in filas_especialidades:
        print(f"Erro: Especialidade '{especialidade}' não suportada.")
        return

    paciente = Paciente(nome_completo, idade, cor_urgencia, especialidade, hora_chegada_str, proxima_sequencia_paciente)
    proxima_sequencia_paciente += 1

    # Chave do heap: (-Prioridade, SequenciaChegada, ObjetoPaciente)
    # Prioridade negativa para que maior prioridade (vermelho=4) saia primeiro.
    # SequenciaChegada garante FIFO quando prioridades são iguais.
    item_heap = (-paciente.prioridade, paciente.sequencia_chegada, paciente)
    heapq.heappush(filas_especialidades[especialidade], item_heap)
    print(f"[{hora_chegada_str}] Paciente {nome_completo} adicionado à {especialidade} com urgência {cor_urgencia}.")

def chamar_proximo_paciente(especialidade):
    """
    Remove e retorna o próximo paciente a ser atendido na especialidade. O(log n_s).
    """
    if especialidade not in filas_especialidades or not filas_especialidades[especialidade]:
        return f"Nenhum paciente na fila de {especialidade}."

    _, _, paciente = heapq.heappop(filas_especialidades[especialidade])

    global ultimo_fim_consulta
    hora_chegada = paciente.hora_chegada

    hora_inicio = max(hora_chegada, ultimo_fim_consulta[especialidade])
    hora_fim = hora_inicio + timedelta(minutes=DURACAO_CONSUTA_MIN)

    ultimo_fim_consulta[especialidade] = hora_fim

    tempo_espera = duracao_para_minutos(hora_inicio - hora_chegada)

    return (f"**CHAMANDO:** {paciente.nome_completo} ({paciente.cor_urgencia}) em {especialidade}. "
            f"Chegada: {hora_chegada.strftime('%H:%M')}, Início: {hora_inicio.strftime('%H:%M')}, "
            f"Fim: {hora_fim.strftime('%H:%M')}. Espera: {max(0, tempo_espera):.0f} min.")

def prever_agenda(especialidade):
    """
    Simula agendamento sequencial para fila atual. O(n_s).
    """
    if especialidade not in filas_especialidades or not filas_especialidades[especialidade]:
        return f"Fila de {especialidade} está vazia."

    copia_fila = filas_especialidades[especialidade][:]

    inicio_simulacao = ultimo_fim_consulta[especialidade]

    heap_temporario = copia_fila[:]
    resultados = []

    while heap_temporario:
        _, _, paciente = heapq.heappop(heap_temporario)
        hora_chegada = paciente.hora_chegada
        hora_inicio = max(hora_chegada, inicio_simulacao)
        hora_fim = hora_inicio + timedelta(minutes=DURACAO_CONSUTA_MIN)
        inicio_simulacao = hora_fim

        resultados.append(
            f"-> {paciente.nome_completo} ({paciente.cor_urgencia}): Chegada {hora_chegada.strftime('%H:%M')}, "
            f"Início Previsto: {hora_inicio.strftime('%H:%M')}, Fim: {hora_fim.strftime('%H:%M')}"
        )

    return "\n".join(resultados)

def tempo_medio_espera(especialidade, hora_atual_str):
    """
    Calcula tempo médio de espera dos pacientes na fila. O(n_s).
    """
    if especialidade not in filas_especialidades or not filas_especialidades[especialidade]:
        return f"Nenhum paciente na fila de {especialidade}."

    hora_atual = string_para_datahora(hora_atual_str)

    copia_fila = filas_especialidades[especialidade][:]
    inicio_simulacao = ultimo_fim_consulta[especialidade]

    tempos_espera = []
    heap_temporario = copia_fila[:]

    while heap_temporario:
        _, _, paciente = heapq.heappop(heap_temporario)
        hora_chegada = paciente.hora_chegada
        hora_inicio = max(hora_chegada, inicio_simulacao)
        hora_fim = hora_inicio + timedelta(minutes=DURACAO_CONSUTA_MIN)
        inicio_simulacao = hora_fim

        minutos_espera = duracao_para_minutos(hora_inicio - hora_chegada)
        tempos_espera.append(max(0, minutos_espera))

    if not tempos_espera:
        return f"Nenhum paciente para calcular tempo de espera em {especialidade}."

    media_espera = sum(tempos_espera) / len(tempos_espera)
    return (f"Tempo médio de espera previsto em {especialidade}: {media_espera:.1f} minutos para a fila atual.")

def mostrar_fila(especialidade):
    """
    Exibe a fila em ordem de atendimento (ordem do heap). O(n_s log n_s).
    """
    if especialidade not in filas_especialidades or not filas_especialidades[especialidade]:
        return f"Fila de {especialidade} está vazia."

    fila_ordenada = sorted(filas_especialidades[especialidade], key=lambda x: x[0:2])

    display = [f"--- Fila: {especialidade} ({len(fila_ordenada)} pacientes) ---"]

    for idx, (_, _, paciente) in enumerate(fila_ordenada):
        display.append(
            f"{idx+1}. {paciente.nome_completo}, {paciente.idade} anos, {paciente.cor_urgencia} "
            f"(Chegada: {paciente.hora_chegada.strftime('%H:%M')})"
        )

    return "\n".join(display)

# --- Exemplo de Teste ---

print("--- 1. Registro de Pacientes ---")
adicionar_paciente('Maria Tagliatte', 34, 'laranja', 'Ortopedia', '08:05')
adicionar_paciente('João Santana', 62, 'amarelo', 'Clínica Geral', '08:07')
adicionar_paciente('Ana Beatriz', 5, 'vermelho', 'Pediatria', '08:09')
adicionar_paciente('Carla Mendonça', 41, 'azul', 'Ortopedia', '08:10')
adicionar_paciente('Pedro Alencar', 58, 'laranja', 'Cardiologia', '08:12')
adicionar_paciente('Rui Barros', 29, 'amarelo', 'Ortopedia', '08:14')
adicionar_paciente('Eva Duarte', 70, 'laranja', 'Cardiologia', '08:15')
adicionar_paciente('Lucas Silva', 22, 'azul', 'Clínica Geral', '08:18')

print("\n--- 2. Ordem de Atendimento (Mostrar Fila) ---")
print(mostrar_fila('Ortopedia'))
print(mostrar_fila('Cardiologia'))
print(mostrar_fila('Pediatria'))

print("\n--- 3. Previsão de Agenda ---")
print(f"\n--- Agenda Ortopedia ---")
print(prever_agenda('Ortopedia'))
print(f"\n--- Agenda Clínica Geral ---")
print(prever_agenda('Clínica Geral'))
print(f"\n--- Agenda Cardiologia ---")
print(prever_agenda('Cardiologia'))

print("\n--- 4. Tempo Médio de Espera ---")
print(tempo_medio_espera('Ortopedia', '08:30'))
print(tempo_medio_espera('Cardiologia', '08:30'))

print("\n--- 5. Chamar Próximo Paciente ---")
print(chamar_proximo_paciente('Ortopedia'))  # Maria (laranja)
print(chamar_proximo_paciente('Ortopedia'))  # Rui (amarelo) - Início: 08:25 (fim de Maria)

print("\n--- 6. Nova Agenda Após 2 Chamadas ---")
print(f"\n--- Agenda Ortopedia Atualizada (Último fim: {ultimo_fim_consulta['Ortopedia'].strftime('%H:%M')}) ---")
print(prever_agenda('Ortopedia'))  # Carla (azul) início 08:45

print(chamar_proximo_paciente('Pediatria'))  # Ana (vermelho)

--- 1. Registro de Pacientes ---
[08:05] Paciente Maria Tagliatte adicionado à Ortopedia com urgência laranja.
[08:07] Paciente João Santana adicionado à Clínica Geral com urgência amarelo.
[08:09] Paciente Ana Beatriz adicionado à Pediatria com urgência vermelho.
[08:10] Paciente Carla Mendonça adicionado à Ortopedia com urgência azul.
[08:12] Paciente Pedro Alencar adicionado à Cardiologia com urgência laranja.
[08:14] Paciente Rui Barros adicionado à Ortopedia com urgência amarelo.
[08:15] Paciente Eva Duarte adicionado à Cardiologia com urgência laranja.
[08:18] Paciente Lucas Silva adicionado à Clínica Geral com urgência azul.

--- 2. Ordem de Atendimento (Mostrar Fila) ---
--- Fila: Ortopedia (3 pacientes) ---
1. Maria Tagliatte, 34 anos, laranja (Chegada: 08:05)
2. Rui Barros, 29 anos, amarelo (Chegada: 08:14)
3. Carla Mendonça, 41 anos, azul (Chegada: 08:10)
--- Fila: Cardiologia (2 pacientes) ---
1. Pedro Alencar, 58 anos, laranja (Chegada: 08:12)
2. Eva Duarte, 70 anos, laran